In [ ]:
#pip install line_profiler

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
import sys
from IPython.display import clear_output
import heapq
import pandas as pd
import time
import csv

from google.colab import drive
import os

#from line_profiler import LineProfiler

In [ ]:
# Function to read the .turb files

def read_turb_file(filename):
  nodes = []
  with open(filename, 'r') as file:
      for line in file:
          # Split each line into x, y, and power
          x, y, power = map(float, line.split())

          # Store the data in a list
          nodes.append([x,y,power])

  return nodes

# Function to read the .cable files

def read_cables_file(filename):
  cables = []
  with open(filename, 'r') as file:

      cable_type = 1

      for line in file:

          # Split each line into x, y, and power
          capacity, price, availability = map(float, line.split())

          capacity = int(capacity)
          availability = int(availability)

          # Store the data in a list
          cables.append([cable_type,capacity,price,availability])

          cable_type += 1

  return cables

# Functions to identify crossing edges in pre-processing step
def do_lines_cross(p1, p2, p3, p4):
    def ccw(A, B, C):
        return (C[1] - A[1]) * (B[0] - A[0]) > (B[1] - A[1]) * (C[0] - A[0])

    return ccw(p1, p3, p4) != ccw(p2, p3, p4) and ccw(p1, p2, p3) != ccw(p1, p2, p4)

def find_crossing_edges(nodes):
    crossing_edges = {}

    for i in range(len(nodes)):
        for j in range(i + 1, len(nodes)):
            for k in range(len(nodes)):
                for l in range(k + 1, len(nodes)):
                    if i != k and i != l and j != k and j != l:
                        x1, y1, power = nodes[i]
                        x2, y2, power = nodes[j]
                        x3, y3, power = nodes[k]
                        x4, y4, power = nodes[l]
                        if do_lines_cross((x1, y1), (x2, y2), (x3, y3), (x4, y4)):
                            edge1 = (i, j)
                            edge2 = (k, l)
                            if edge1 not in crossing_edges:
                                crossing_edges[edge1] = []
                            if edge2 not in crossing_edges:
                                crossing_edges[edge2] = []
                            crossing_edges[edge1].append(edge2)
                            crossing_edges[edge2].append(edge1)

    return crossing_edges




# Function to create a distance matrix

def create_dist_matrix(nodes):
  n = len(nodes)
  A = np.zeros((n,n))
  for i in range(n):
    for j in range(n):
      if nodes[i]!=nodes[j] and A[i][j]==0:
        A[i][j] =  math.sqrt( (nodes[j][0] - nodes[i][0])**2 + (nodes[j][1] - nodes[i][1])**2 )
        A[j][i] = A[i][j]

  return A

# Function to create a list contaning the angles with respect to substation

def compute_angles(nodes):
  n = len(nodes)
  angles = []
  angles.append(0.00)

  for i in range(1,n):
    # Use unit vector pointing rightwards as reference
    ax = 1.0
    ay = 0.0
    # Create vector b from substation to a given node
    bx = nodes[i][0] - nodes[0][0]
    by = nodes[i][1] - nodes[0][1]
    # Calculate the angle using the arctan2 function
    angle_radians = math.atan2(by, bx) - math.atan2(ay, ax)
    # Normalize the angle to the range [0, 2*pi]
    angle = angle_radians % (2 * math.pi)
    angles.append(angle)
  return angles

# Function to group together nodes

def node_grouping(nodes,indexed_angles,starting_turb,TPG):

  # Shift the list order based on the chosen starting node
  shifted_angles = []
  counter = 0
  n = len(indexed_angles)
  # Find the list position of the starting node
  for k in range(n):
    if indexed_angles[k][0] == starting_turb:
      i = k
  while counter < n:
    shifted_angles.append(indexed_angles[i])
    i = (i + 1) % n
    counter += 1
  indexed_angles = shifted_angles
  # Create the turbine groups
  groups = []
  while indexed_angles:
    group = []
    for i in range(TPG):
      if indexed_angles:
        item = indexed_angles.pop(0)
        node = item[0]
        group.append(node)
    groups.append(group)
  return groups


# Function to set the costs between nodes of different groups to infinity

def segregate(dist_matrix, groups):

  cost_matrix = np.copy(dist_matrix)
  n = cost_matrix.shape[0]
  g = len(groups)
  for i in range(1,n):
    for j in range(g):
      if i not in groups[j]:
        for k in groups[j]:
          cost_matrix[i][k] = float('inf')
  return cost_matrix


# Prim's algorithm to compute the MST
"""
def prim(A):
  n = A.shape[0]
  # Define an array B to hold the final adjacency matrix
  B = np.zeros((n,n))
  # Define the initial cost of the MST
  cost = 0.0
  # Define the set of nodes in the tree and its complement
  T = [0]
  Tc = []
  for i in range(1,n):
    Tc.append(i)
  # Add the node with minimum weight that links to the tree
  while Tc:
    min_weight = float('inf')
    for i in T:
      for j in Tc:
        if A[i][j] < min_weight:
          node_index = i
          neighbor_index = j
          min_weight = A[i][j]
    T.append(neighbor_index)
    Tc.remove(neighbor_index)
    B[node_index][neighbor_index] = 1
    B[neighbor_index][node_index] = 1
    cost = cost + min_weight

  return B
"""

def prim(A):
    n = A.shape[0]
    B = np.zeros((n,n))
    # Initialize the MST edges and total cost
    edges = []
    cost = 0.0
    # Initialize the set of nodes in the tree and its complement
    in_tree = {0}
    not_in_tree = set(range(1, n))
    # Add the neighbors of the first node to the priority queue
    for j in range(1, n):
        heapq.heappush(edges, (A[0][j], 0, j))

    while edges:
        if len(in_tree) == n:
          break
        # Get the minimum edge from the priority queue
        weight, node_index, neighbor_index = heapq.heappop(edges)
        # If the neighbor is not in the tree, add it to the tree
        if neighbor_index in not_in_tree:
            in_tree.add(neighbor_index)
            not_in_tree.remove(neighbor_index)
            # Update the total cost
            cost += weight
            # Add the edge to the MST
            B[node_index][neighbor_index] = 1
            B[neighbor_index][node_index] = 1
            # Add the neighbors of the newly added node to the priority queue
            for j in not_in_tree:
                heapq.heappush(edges, (A[neighbor_index][j], neighbor_index, j))

    return B



# Modified implementation of Depth First Search to compute the flows

def DFS(A,v,visited,next,leaves,flows):

  visited[v]=True

  if (v==0):
    next[v]=0

  n = A.shape[0]

  neighbor_counter = 0

  total_flow = 0

  for neighbor in range(n):

    if A[v][neighbor]==1 and not visited[neighbor]:
      neighbor_counter += 1
      next[neighbor]=v
      edge = (neighbor,v)
      if edge not in flows:
        flows[edge] = DFS(A,neighbor,visited,next,leaves,flows)
      total_flow += flows[edge]

  if neighbor_counter == 0:
    leaves.append(v)
    edge = (v,next[v])
    flows[edge] = 1
    return 1

  if (v == 0):
    return total_flow
  else:
    return 1 + total_flow

# Function to compute the prices associated with a choice of cable types

def compute_price(distance_matrix,flows,cable_types):
  info_dict = {}
  total_price = 0.0
  n_types = len(cable_types)

  # Compute total price based on cost function
  for (i,j),f in flows.items():
    # Use bool variable to keep track of overflow
    overflow = True
    # Select appropriate cable type
    for cable_type in cable_types:
      if cable_type[1]>=f:
        overflow = False
        info_dict[(i,j)] = (cable_type[0],overflow)
        total_price += cable_type[2]*distance_matrix[i][j]
        break
    if overflow==True:
      info_dict[(i,j)]= (cable_types[n_types-1][0],overflow)
      total_price += cable_types[n_types-1][2]*distance_matrix[i][j]
      total_price += float('inf')
      print("OVERFLOW HAPPENED")
  return info_dict, total_price


#%load_ext line_profiler

def multi_sweep(nodes,cable_types,instance_number):

  # Create the distance matrix
  dist_matrix = create_dist_matrix(nodes)

  # Determine the TPG range
  n = dist_matrix.shape[0]-1
  C = math.ceil(n/cable_types[len(cable_types)-1][1]) +1
  TPG_low = n//C
  TPG_high = cable_types[len(cable_types)-1][1]//1

  # Initialize candidate solution
  best_info_dict = {}
  best_total_price = float('inf')

  # Initialize run counter
  counter = 1
  max = (TPG_high-TPG_low + 1)*n*2

  # Compute angles
  angles = compute_angles(nodes)

  # Create a list of tuples containing the index of the node and its angle
  indexed_angles = [(index, angle) for index, angle in enumerate(angles)]
  indexed_angles.pop(0)
  cclock_indexed_angles = indexed_angles = sorted(indexed_angles, key=lambda x: x[1])
  clock_indexed_angles = indexed_angles = sorted(indexed_angles, key=lambda x: x[1], reverse=True)

  for TPG in range(TPG_low,TPG_high + 1):
    for starting_turb in range(1,n+1):
      for direction in range(0,1+1):
        # Print current status
        """
        clear_output(wait=True)
        print("Instance:",instance_number,"/ 220")
        print("Run:",counter,"/",max)
        """
        # Run instance of Sweep
        if direction == 0:
          groups = node_grouping(nodes,cclock_indexed_angles,starting_turb,TPG)
        if direction == 1:
          groups = node_grouping(nodes,clock_indexed_angles,starting_turb,TPG)

        # Compute flow, select cable types and determine the cost
        cost_matrix = segregate(dist_matrix,groups)
        T = prim(cost_matrix)
        visited = [False]*T.shape[0]
        next = [0]*T.shape[0]
        leaves = []
        flows = {}
        DFS(T,0,visited,next,leaves,flows)
        info_dict, total_price = compute_price(dist_matrix,flows,cable_types)
        if total_price < best_total_price:
          best_info_dict = info_dict.copy()
          best_total_price = total_price

        counter += 1

  return best_info_dict, best_total_price

# Function to plot the results

def plot_results(nodes,info_dict,output_path):

  # Plotting turbines

  for i, (x, y, power) in enumerate(nodes):
      plt.scatter(x, y, color='black')
      plt.text(x + 100, y, f'{i}', fontsize=8, verticalalignment='center')

  # Plotting connections

  for (i, j), (cable_type, _) in info_dict.items():
      xi, yi, _ = nodes[i]
      xj, yj, _ = nodes[j]
      dx, dy = xj - xi, yj - yi
      if cable_type == 1:
        color = 'xkcd:sky blue'
      if cable_type == 2:
        color = 'xkcd:green'
      if cable_type == 3:
        color = 'xkcd:red'
      plt.arrow(xi, yi, dx, dy,fc = color, ec = color,length_includes_head=True)

  # Add legend for cable types

  plt.legend(handles=[
      plt.Line2D([0], [0], color='xkcd:sky blue', lw=2, label='Type 5'),
      plt.Line2D([0], [0], color='xkcd:green', lw=2, label='Type 7'),
      plt.Line2D([0], [0], color='xkcd:red', lw=2, label='Type 12')
  ])

  plt.axis('off')
  plt.grid(False)



  # Save the plot as a PNG file
  plt.show()
  #plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
  plt.close()




In [ ]:
# BACKTRACKING METHOD-----------------------------------------------

def branchbound(nodes,cable_types,upper_bound,crossing_edges):

  # Create distance matrix
  n = len(nodes)
  dist = create_dist_matrix(nodes)

  # Sort the nodes according to distance to substation
  sorted_turbs = dist[0]
  sorted_turbs = list(np.argsort(sorted_turbs))
  sorted_turbs.pop(0)

  # Initialize priority queue
  pq = []
  first_turb = sorted_turbs.pop(0)
  first_edge = (first_turb,0)
  first_cost = 0

  T = np.zeros((n,n))
  T[first_turb][0]=1
  T[0][first_turb]=1

  # Initial state: inital cost, initial edges
  heapq.heappush(pq, ( first_cost , [first_edge],T))

  # Define upper bound
  best_cost = upper_bound
  best_solution = None
  best_info_dict = None
  counter = 1

  # Set the time limit to 60 seconds
  time_limit = 60
  start_time = time.time()
  heaplen = len(pq)
  while pq:

    counter+=1

    if (time.time() - start_time) >= time_limit:
      print(counter)
      return best_solution, best_cost, best_info_dict

    cost,connected,T = heapq.heappop(pq)

    turbines_in_tree = set()

    for edge in connected:
      turbines_in_tree.add(edge[0])
      turbines_in_tree.add(edge[1])


    # Check to see if a full solution has been found
    if len(connected) == n-1 :

      visited = [False]*T.shape[0]
      next = [0]*T.shape[0]
      leaves = []
      flows = {}

      # Found a complete solution

      DFS(T,0,visited,next,leaves,flows)
      info_dict, cost = compute_price(dist,flows,cable_types)

      if cost >= best_cost:

        continue

      if cost < best_cost:
        best_cost = cost
        best_solution = connected
        best_info_dict = info_dict

      continue


    for i in range(n):
        if i not in turbines_in_tree:
          for j in turbines_in_tree:

            # Check for crossing---------------
            edge_does_cross = False
            for edge in connected:
              if edge in crossing_edges:
                if (i,j) in crossing_edges[edge]:
                  edge_does_cross = True
            if edge_does_cross == True:
              continue
            #----------------------------------

            new_connected = connected + [(i,j)]

            T[i][j] = 1
            T[j][i] = 1

            # Compute cost estimate
            new_cost = cost + dist[i][j]*cable_types[0][0]


            # Push the partial solution into the priority queue
            heapq.heappush(pq, (new_cost , new_connected, T.copy() ) )

            T[i][j] = 0
            T[j][i] = 0


  print(counter)

  return best_solution, best_cost, best_info_dict



In [ ]:

"""
filename_turb = "A.turb"
filename_cables = "test.cable"

nodes = read_turb_file(filename_turb)
cable_types = read_cables_file(filename_cables)
crossing_edges = find_crossing_edges(nodes)

info_dict, sweep_cost = multi_sweep(nodes,cable_types,1)
print("SWEEP COST:",sweep_cost)
upper_bound = sweep_cost + 0.001
bs,bc,bi = branchbound(nodes,cable_types,upper_bound,crossing_edges)
print("BRANCHBOUND COST:", bc)
print(bi)
"""

In [ ]:
input_path = '/content/'

turb_files = os.listdir(input_path)
turb_files.remove('.ipynb_checkpoints')
turb_files.remove('sample_data')
turb_files.remove('.config')
turb_files.remove('test.cable')
turb_files.sort()

cable_files = 'test.cable'

# Define the directory name
directory = "/content/results"

# Create the directory
os.makedirs(directory, exist_ok=True)
turb_files.remove('results')


# Create instance dictionary
instance_dict = {}

# Loop through instances
for instance in turb_files:
  # Print status
  print("Current instance:",instance)
  # Run Sweep
  filename_turb = instance[:]
  instance = os.path.splitext(instance)[0]
  filename_cables = "test.cable"
  output_path = '/content/results/'+ instance + '_sweep.png'

  nodes = read_turb_file(filename_turb)
  cable_types = read_cables_file(filename_cables)
  crossing_edges = find_crossing_edges(nodes)

  info_dict, sweep_cost = multi_sweep(nodes,cable_types,1)

  plot_results(nodes,info_dict,output_path)

  upper_bound = sweep_cost + 0.001

  # Run Branch and Bound
  output_path = '/content/results/'+ instance + '_exact.png'
  bs, bc, bi = branchbound(nodes,cable_types,upper_bound, crossing_edges)
  bb_cost = bc
  plot_results(nodes,bi,output_path)

  # Compute gap
  gap = ((sweep_cost - bc)/bc)*100
  instance_dict[instance] = (sweep_cost,bb_cost,gap)

# Create .csv table

csv_folder_path = '/content/results/results_table.csv'
# Write dictionary to CSV file
with open(csv_folder_path, 'w', newline='') as csvfile:
    fieldnames = ['Instance', 'Sweep Cost', 'Exact Cost', 'Gap']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for key, value in instance_dict.items():
        writer.writerow({'Instance': key, 'Sweep Cost': value[0], 'Exact Cost': value[1], 'Gap': value[2]})


print("CSV file has been created successfully.")